## Подготовка данных для занятия

In [2]:
!pip install pymorphy2

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13709 sha256=6b06507e6e7b21ddd0a6f32212ea903bdce31fa13b40816cfe2ebd53a2d6ec0b
  Stored in directory: c:\users\rookie\appdata\local\pip\cache\wheels\56\ea\58\ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [3]:
import re

import pandas as pd
import pymorphy2

text_df = pd.read_csv("./data/content_description.csv", sep='\t')
text_df.head()

# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)
# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

,doc_id,word,dummy
6,0,в,10
29,0,и,7
37,0,который,5
111,0,шерлок,4
77,0,с,4
84,0,сериал,4
75,0,риколетти,3
31,0,история,2
21,0,для,2
49,0,о,2


# 10.3  Домашняя работа

10.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процен документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [16]:
# -- ВАШ КОД ЗДЕСЬ --
import numpy as np

doc_frequency = pd.DataFrame({
    "word": tokens,
    "df": np.zeros_like(tokens)
})

doc_id_wordsets = word_count_df.groupby("doc_id").agg({"word": lambda x: set(x)})["word"].values
#Кол-во документов
n = len(word_count_df["doc_id"].unique())

for i, word in enumerate(tokens):
    m = sum([1 if word in s else 0 for s in doc_id_wordsets])
    doc_frequency["df"].iloc[i]= m / n * 100.
    
doc_frequency.head()

,word,df
0,хороший,12.5
1,подарок,25
2,который,50
3,только,25
4,можно,37.5


10.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [25]:
# -- ВАШ КОД ЗДЕСЬ --
raw_text = text_df.description.values[3]
# регулярка - её нужно поправить
reg_expr = r'(?<!\. )([А-Я]\w+) ([А-Я]\w+)' # исправил чтобы не было Теперь Тед
# компилируем регулярное выражение
reg_expr_compiled = re.compile(reg_expr)
# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    print(g)

('Стивена', 'Фрая')
('Джоном', 'Дженксом')
('Теда', 'Уоллеса')


10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [37]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}

nested_genres =[[(i, j) for j in genre_dict[i]] for i in genre_dict]
# переводим из словаря к более удобному виду
flatten_genres = list(itertools.chain(*nested_genres))
# создаём DataFrame
genres_df = pd.DataFrame(flatten_genres, columns = ['genre', 'character_word'])


# -- ВАШ КОД ЗДЕСЬ --
genres_df.columns = ["genre", "word"]
merged = pd.merge(word_count_df, genres_df, how="inner", on="word")
merged

,doc_id,word,dummy,genre
0,0,загадочный,1,детектив
1,0,разгадать,1,детектив
2,0,тайна,1,детектив
3,0,ужас,1,триллер
4,6,ужас,3,триллер
5,1,милый,1,сказка
6,2,выбор,1,мелодрама
7,2,позор,1,мелодрама
8,3,сатирический,1,комедия
9,4,авантюрный,1,комедия


In [39]:
doc_to_genres = merged.groupby("doc_id").agg({"genre": lambda x: set(x)})
doc_to_genres

,genre
doc_id,
0,"{детектив, триллер}"
1,{сказка}
2,{мелодрама}
3,{комедия}
4,{комедия}
6,{триллер}
